# Common setup

## Install dependencies

In [42]:
!pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


## Imports

In [43]:
import argparse
import os
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, logging
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

## Set logging

In [44]:
# Silences "Some weights were not initialized from the model checkpoint" warning
logging.set_verbosity_error()

## Input files

In [45]:
input_files=["/home/levi/rhel-8-docs/rhel-9/modules/performance/con_grouping-vdo-threads-across-numa-nodes.adoc", "/home/levi/rhel-8-docs/rhel-9/modules/performance/proc_analyzing-vdo-performance-with-perf.adoc", "/home/levi/rhel-8-docs/rhel-9/modules/performance/proc_analyzing-vdo-performance-with-sar.adoc", "/home/levi/rhel-8-docs/rhel-9/modules/performance/proc_analyzing-vdo-performance-with-top.adoc", "/home/levi/rhel-8-docs/rhel-9/modules/performance/proc_configuring-the-cpu-affinity.adoc", "/home/levi/rhel-8-docs/rhel-9/modules/performance/proc_increasing-block-cache-size.adoc", "/home/levi/rhel-8-docs/rhel-9/modules/performance/proc_optimizing-cpu-frequency-scaling-for-vdo-performance.adoc", "/home/levi/rhel-8-docs/rhel-9/modules/performance/proc_speeding-up-discard-operations.adoc", "/home/levi/rhel-8-docs/rhel-9/modules/performance/ref_vdo-thread-types.adoc"]

## Define main function

In [46]:
def main(input_files):
    # parser = argparse.ArgumentParser(description="Abstractive Text Summarization Script")
    # parser.add_argument("input_files", nargs='+', help="Paths to the input text files")

    # args = parser.parse_args()
    # input_files = args.input_files

    for input_file_path in input_files:
        # Check if the file exists
        if not os.path.exists(input_file_path):
            print(f"Error: File '{input_file_path}' does not exist.")
            continue

        # Read input text from the file
        try:
            with open(input_file_path, "r", encoding="utf-8") as file:
                input_text = file.read()

            # Check if the file is empty
            if not input_text.strip():
                print(f"Error: File '{input_file_path}' is empty.")
                continue
        
        # Check if the file is corrupt
        except Exception as e:
            print(f"Error reading file '{input_file_path}': {e}")
            continue
        
        summary_dict = abstractive_summarization(input_file_path)
        
        for filepath, summary in summary_dict.items():
            print(f"\nFilename: {filepath}\nSummary: {summary}")


# Summarization function Pegasus

In [47]:
def abstractive_summarization(file):
    # Load pre-trained model and tokenizer only once
    model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
    tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum", model_max_length=1024)

    # Tokenize and generate summary
    with open(file, "r") as f:
        text = f.read()
        
        inputs = tokenizer("summarize: " + text, return_tensors="pt", padding="longest", truncation=True)
        summary_ids = model.generate(inputs["input_ids"], max_length=150, length_penalty=2.0, num_beams=4, early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        return {file: summary}
    
main(input_files)


Filename: /home/levi/rhel-8-docs/rhel-9/modules/performance/con_grouping-vdo-threads-across-numa-nodes.adoc
Summary: VDO-related work on the same NUMA node can cause memory contention if threads are spread across nodes.

Filename: /home/levi/rhel-8-docs/rhel-9/modules/performance/proc_analyzing-vdo-performance-with-perf.adoc
Summary: The following is a list of the most frequently asked questions about the latest version of the Linux kernel.

Filename: /home/levi/rhel-8-docs/rhel-9/modules/performance/proc_analyzing-vdo-performance-with-sar.adoc
Summary: The following is a list of the most common driver problems reported by users of Windows 10.

Filename: /home/levi/rhel-8-docs/rhel-9/modules/performance/proc_analyzing-vdo-performance-with-top.adoc
Summary: The top utility displays statistics on the performance of VDO threads running on a CPU.

Filename: /home/levi/rhel-8-docs/rhel-9/modules/performance/proc_configuring-the-cpu-affinity.adoc
Summary: The following instructions will sho